<a href="https://colab.research.google.com/github/Vipinanalyst/Vipinanalyst/blob/main/VNL_Data_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas pyreadstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 22.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Read the .sav file using pandas
df = pd.read_spss('total VNL DATA.sav')

# Display the DataFrame
df.head()


,NO,RESULT,ATTACK,BLOCK,SERVE,ERROR
0,1.0,won,37.0,10.0,9.0,20.0
1,2.0,won,61.0,9.0,4.0,23.0
2,3.0,won,59.0,9.0,7.0,24.0
3,4.0,won,53.0,13.0,8.0,29.0
4,5.0,won,66.0,11.0,6.0,28.0


In [ ]:
df = df.drop('NO',axis=1)

In [ ]:
df.head()

,RESULT,ATTACK,BLOCK,SERVE,ERROR
0,won,37.0,10.0,9.0,20.0
1,won,61.0,9.0,4.0,23.0
2,won,59.0,9.0,7.0,24.0
3,won,53.0,13.0,8.0,29.0
4,won,66.0,11.0,6.0,28.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   RESULT  208 non-null    category
 1   ATTACK  208 non-null    float64 
 2   BLOCK   208 non-null    float64 
 3   SERVE   208 non-null    float64 
 4   ERROR   208 non-null    float64 
dtypes: category(1), float64(4)
memory usage: 6.9 KB


In [ ]:
df['RESULT'] = df['RESULT'].map({'won': 1, 'loss': 0})

In [ ]:
df.head()

,RESULT,ATTACK,BLOCK,SERVE,ERROR
0,1,37.0,10.0,9.0,20.0
1,1,61.0,9.0,4.0,23.0
2,1,59.0,9.0,7.0,24.0
3,1,53.0,13.0,8.0,29.0
4,1,66.0,11.0,6.0,28.0


In [ ]:
df.isna().sum()

RESULT    0
ATTACK    0
BLOCK     0
SERVE     0
ERROR     0
dtype: int64

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

# Fit and transform the numerical columns in the DataFrame
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

df_normalized.head()

,RESULT,ATTACK,BLOCK,SERVE,ERROR
0,1.0,0.180328,0.588235,0.642857,0.394737
1,1.0,0.573770,0.529412,0.285714,0.473684
2,1.0,0.540984,0.529412,0.500000,0.500000
3,1.0,0.442623,0.764706,0.571429,0.631579
4,1.0,0.655738,0.647059,0.428571,0.605263


In [ ]:
from sklearn.model_selection import train_test_split

X = df_normalized.drop('RESULT', axis=1)
y = df_normalized['RESULT']

# Split the data into training and testing sets (80% train, 20% test in this example)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

In [ ]:
print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.74
Confusion Matrix:
[[19  3]
 [ 8 12]]


In [ ]:
from sklearn.metrics import mean_absolute_error, precision_score, recall_score
mae = mean_absolute_error(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

Mean Absolute Error: 0.26
Precision: 0.80
Recall: 0.60


In [ ]:
ppv = conf_matrix[1, 1] / (conf_matrix[1, 1] + conf_matrix[0, 1]) if (conf_matrix[0, 1] + conf_matrix[1, 1]) != 0 else 0

# Calculate NPV
npv = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[1, 0]) if (conf_matrix[0, 0] + conf_matrix[1, 0]) != 0 else 0

print(f"PPV (Precision): {ppv:.2f}")
print(f"NPV: {npv:.2f}")

PPV (Precision): 0.80
NPV: 0.70


In [ ]:
# Get the coefficients and feature names
coefficients = model.coef_[0]
feature_names = X.columns

# Create a DataFrame to display coefficients and feature names
coefficients_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
})

# Sort the DataFrame by the absolute value of coefficients to see the most important features
coefficients_df['Absolute Coefficient'] = coefficients_df['Coefficient'].abs()
sorted_coefficients_df = coefficients_df.sort_values(by='Absolute Coefficient', ascending=False)

# Display the sorted coefficients
print(sorted_coefficients_df)


  Feature  Coefficient  Absolute Coefficient
2   SERVE     2.017447              2.017447
1   BLOCK     1.599956              1.599956
0  ATTACK     0.836543              0.836543
3   ERROR     0.037227              0.037227


In [ ]:
Precision (Precision_score): Of all the instances predicted as "won" (positive), how many were actually "won" (True Positives)?
 A precision of 0.8 means that, among the instances predicted as "won," 80% were correctly predicted.

Recall (Recall_score): Of all the actual instances of "won," how many did the model correctly predict as "won" (True Positives)?
 A recall of 0.6 means that the model captured 60% of the instances of "won."